In [1]:
import json
import torch
import pickle
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from transformers import AdamW, GPT2Config, GPT2LMHeadModel

In [2]:
json_wikipedia_file = 'text/AA/wiki_00' 
plain_wikipedia_file = 'plain_wikipedia.txt'

Read the json file and write the article title and text into a single text file

In [3]:
with open(plain_wikipedia_file, 'w') as f:
    for line in open(json_wikipedia_file, 'r', encoding='utf-8'):
        article = json.loads(line)
        f.write(article['title'])
        f.write('\n')
        f.write(article['text'])
        f.write('\n')

Prepare the Tokenizer: it is the object that will analyse all the text and build the vocabulary

In [35]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"], vocab_size=5000)

Analyse the text using the tikenizer to get the vocabulary

In [36]:
tokenizer.train(files=[plain_wikipedia_file], trainer=trainer)
tokenizer.save("tokenizer.json")

In [3]:
# tokenizer = Tokenizer.from_file("tokenizer.json")

In [37]:
sentence_to_complete = "ប្រាសាទ អង្គរវត្ត"

output = tokenizer.encode(sentence_to_complete)
print(output.ids)
print(output.tokens)

[3632, 4960, 1138, 3391]
['ប្រាសាទ', ' អង្គ', 'រ', 'វត្ត']


In [39]:
print(tokenizer.decode(output.ids, skip_special_tokens=False))

ប្រាសាទ  អង្គ រ វត្ត


In [40]:
tokenized_texts = []
with open(plain_wikipedia_file, 'r', encoding='utf-8') as file:
    for line in tqdm(file):
        tokenized_line = tokenizer.encode(line.strip()).ids
        if tokenized_line:
            tokenized_texts.append(tokenized_line[:32])

170392it [00:11, 14380.72it/s]


In [46]:
class TextDataset(Dataset):
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts)

    def __getitem__(self, idx):
        return torch.tensor(self.tokenized_texts[idx])

def collate_batch(batch):
    # Pad the sequences in the batch
    batch_padded = pad_sequence([sequence for sequence in batch], 
                                batch_first=True, padding_value=tokenizer.token_to_id("[PAD]"))
    # Create attention masks
    attention_masks = torch.zeros(batch_padded.shape, dtype=torch.long)
    attention_masks[batch_padded != tokenizer.token_to_id("[PAD]")] = 1

    return batch_padded, attention_masks


dataset = TextDataset(tokenized_texts)
data_loader = DataLoader(dataset, batch_size=128, shuffle=True, collate_fn=collate_batch)

In [47]:
config = GPT2Config(vocab_size=len(tokenizer.get_vocab()), n_positions=32, n_layer=4, n_head=4, n_embed=64, pad_token_id = tokenizer.token_to_id("[PAD]"))
model = GPT2LMHeadModel(config)
device = torch.device("cuda")
model.to(device)

# Set up optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [48]:
input_text = sentence_to_complete
input_ids = torch.tensor([tokenizer.encode(input_text).ids]).to(device)
attention_mask = torch.ones_like(input_ids).to(device)
max_length = 32  # Generating 5 additional tokens
model.eval()
output = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, num_beams=5, num_return_sequences=5).to(device)
for prediction in output:
    predicted_text = tokenizer.decode(prediction.tolist(), skip_special_tokens=True)
    print("Predicted text:", predicted_text)

Predicted text: ប្រាសាទ  អង្គ រ វត្ត I I I I b b b b b b ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី 
Predicted text: ប្រាសាទ  អង្គ រ វត្ត I I I I b b b b b b ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ຸ
Predicted text: ប្រាសាទ  អង្គ រ វត្ត I I I I b b b b b b ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  គ្រប់គ្រ
Predicted text: ប្រាសាទ  អង្គ រ វត្ត I I I I b b b b b b ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  
Predicted text: ប្រាសាទ  អង្គ រ វត្ត I I I I b b b b b b ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ី  ଼


In [49]:
model.train()
batch_id = 0
max_epoch = 10
for epoch in range(max_epoch):
    for batch, attention_mask in data_loader:
        optimizer.zero_grad()
        input_ids = batch.to(device)
        attention_mask = attention_mask.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        if batch_id % 100 == 0:
            print(f'Epoch: {epoch}, Batch: {batch_id}/{len(data_loader)*max_epoch}, Loss: {loss.item()}')
        batch_id += 1
        # if batch_id % 500 == 0:
        #     break

Epoch: 0, Batch: 0/12970, Loss: 8.368127822875977
Epoch: 0, Batch: 100/12970, Loss: 4.9140119552612305
Epoch: 0, Batch: 200/12970, Loss: 4.436372756958008
Epoch: 0, Batch: 300/12970, Loss: 4.169074535369873
Epoch: 0, Batch: 400/12970, Loss: 3.6056904792785645
Epoch: 0, Batch: 500/12970, Loss: 3.4483678340911865
Epoch: 0, Batch: 600/12970, Loss: 3.5918898582458496
Epoch: 0, Batch: 700/12970, Loss: 3.2188501358032227
Epoch: 0, Batch: 800/12970, Loss: 3.2799620628356934
Epoch: 0, Batch: 900/12970, Loss: 3.091048240661621
Epoch: 0, Batch: 1000/12970, Loss: 3.4663119316101074
Epoch: 0, Batch: 1100/12970, Loss: 3.3963613510131836
Epoch: 0, Batch: 1200/12970, Loss: 2.946043014526367
Epoch: 1, Batch: 1300/12970, Loss: 2.806199550628662
Epoch: 1, Batch: 1400/12970, Loss: 2.561195135116577
Epoch: 1, Batch: 1500/12970, Loss: 2.9467434883117676
Epoch: 1, Batch: 1600/12970, Loss: 3.0001707077026367
Epoch: 1, Batch: 1700/12970, Loss: 2.7308874130249023
Epoch: 1, Batch: 1800/12970, Loss: 2.9536268711

In [53]:
sentence_to_complete = "ប្រាសាទ អង្គរវត្ត"
input_text = sentence_to_complete
input_ids = torch.tensor([tokenizer.encode(input_text).ids]).to(device)
attention_mask = torch.ones_like(input_ids).to(device)
max_length = 32  # Generating 5 additional tokens
model.eval()
output = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, num_beams=10, num_return_sequences=10).to(device)
for prediction in output:
    predicted_text = tokenizer.decode(prediction.tolist(), skip_special_tokens=True)
    print("Predicted text:", predicted_text)

Predicted text: ប្រាសាទ  អង្គ រ វត្ត  ជា ស្ថាប ត ្យ កម្ម ខ្មែរ ដែល កសាង ឡើង ដោយ ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧  ក្នុង រាជ ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧ ។  ប្រាសាទ នេះ កសាង ឡើង ក្នុង
Predicted text: ប្រាសាទ  អង្គ រ វត្ត  ជា ស្ថាប ត ្យ កម្ម ខ្មែរ  ដែល កសាង ឡើង ដោយ ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧  ក្នុង រាជ ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧ ។  ប្រាសាទ នេះ កសាង ឡើង ក្នុង
Predicted text: ប្រាសាទ  អង្គ រ វត្ត  ជា ស្ថាប ត ្យ កម្ម ខ្មែរ ដែល កសាង ឡើង ដោយ ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧  ក្នុង រាជ ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧ ។  ប្រាសាទ នេះ កសាង ឡើង នៅ
Predicted text: ប្រាសាទ  អង្គ រ វត្ត  ជា ស្ថាប ត ្យ កម្ម ខ្មែរ  ដែល កសាង ឡើង ដោយ ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧  ក្នុង រាជ ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧ ។  ប្រាសាទ នេះ កសាង ឡើង នៅ
Predicted text: ប្រាសាទ  អង្គ រ វត្ត  ជា ស្ថាប ត ្យ កម្ម ខ្មែរ  ដែល កសាង ឡើង ដោយ ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧  ក្នុង រាជ ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧ ។  ប្រាសាទ នេះ ត្រូវបាន កសាង ឡើង
Predicted text: ប្រាសាទ  អង្គ រ វត្ត  ជា ស្ថាប ត ្យ កម្ម ខ្មែរ ដែល កសាង ឡើង ដោយ ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧  ក្នុង រាជ ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧ ។  ប្រាសាទ នេះ ត

In [54]:
sentence_to_complete = "ប្រាសាទ"
input_text = sentence_to_complete
input_ids = torch.tensor([tokenizer.encode(input_text).ids]).to(device)
attention_mask = torch.ones_like(input_ids).to(device)
max_length = 32  # Generating 5 additional tokens
model.eval()
output = model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, num_beams=10, num_return_sequences=10).to(device)
for prediction in output:
    predicted_text = tokenizer.decode(prediction.tolist(), skip_special_tokens=True)
    print("Predicted text:", predicted_text)

Predicted text: ប្រាសាទ ស ំប ូរ ព្រៃ គ ុក មាន ទីតាំង ស្ថិតនៅក្នុង ភូមិ ស ំប ូរ  ឃុំ ស ំប ូរ  ស្រុក ប្រាសាទ ស ំប ូរ  ខេត្ត កំពង់ ធំ ។  ប្រាសាទ នេះ កសាង ឡើង ក្នុង
Predicted text: ប្រាសាទ ស ំប ូរ ព្រៃ គ ុក មាន ទីតាំង ស្ថិតនៅក្នុង ភូមិ ស ំប ូរ  ឃុំ ស ំប ូរ  ស្រុក ប្រាសាទ ស ំប ូរ  ខេត្ត កំពង់ ធំ ។  ប្រាសាទ នេះមាន ច ម្ ងាយ
Predicted text: ប្រាសាទ ស ំប ូរ ព្រៃ គ ុក មាន ទីតាំង ស្ថិតនៅក្នុង ភូមិ ស ំប ូរ  ឃុំ ស ំប ូរ  ស្រុក ប្រាសាទ ស ំប ូរ  ខេត្ត កំពង់ ធំ ។  ប្រាសាទ នេះ ស្ថិតនៅ ច ម្
Predicted text: ប្រាសាទ ស ំប ូរ ព្រៃ គ ុក មាន ទីតាំង ស្ថិតនៅ ភូមិ ស ំប ូរ  ឃុំ ស ំប ូរ  ស្រុក ប្រាសាទ ស ំប ូរ  ខេត្ត កំពង់ ធំ ។  ប្រាសាទ នេះ កសាង ឡើង ក្នុង
Predicted text: ប្រាសាទ ស ំប ូរ ព្រៃ គ ុក មាន ទីតាំង ស្ថិតនៅក្នុង ភូមិ ស ំប ូរ  ឃុំ ស ំប ូរ  ស្រុក ប្រាសាទ ស ំប ូរ  ខេត្ត កំពង់ ធំ ។  ប្រាសាទ នេះ កសាង ឡើង នៅ
Predicted text: ប្រាសាទ នាង ខ្ម ៅ មាន ទីតាំង ស្ថិតនៅក្នុង ភូមិ ស ំប ូរ  ឃុំ ស ំប ូរ  ស្រុក ប្រាសាទ ស ំប ូរ  ខេត្ត កំពង់ ធំ ។  ប្រាសាទ នេះ កសាង ឡើង ក្នុង សម័យ ច េន
Predicted text: ប្រាសាទ ស ំប ូរ ព្រៃ គ ុក មាន

In [ ]:
model.save_pretrained("gpt2-wikipedia-khmer")

In [56]:
new_model = GPT2LMHeadModel.from_pretrained("gpt2-wikipedia-khmer")

In [57]:
sentence_to_complete = "ប្រាសាទ"
input_text = sentence_to_complete
input_ids = torch.tensor([tokenizer.encode(input_text).ids])
attention_mask = torch.ones_like(input_ids)
max_length = 32  # Generating 5 additional tokens
new_model.eval()
output = new_model.generate(input_ids, attention_mask=attention_mask, max_length=max_length, num_beams=10, num_return_sequences=10).to(device)
for prediction in output:
    predicted_text = tokenizer.decode(prediction.tolist(), skip_special_tokens=True)
    print("Predicted text:", predicted_text)

Predicted text: ប្រាសាទ ស ំប ូរ ព្រៃ គ ុក មាន ទីតាំង ស្ថិតនៅក្នុង ភូមិ ស ំប ូរ  ឃុំ ស ំប ូរ  ស្រុក ប្រាសាទ ស ំប ូរ  ខេត្ត កំពង់ ធំ ។  ប្រាសាទ នេះ កសាង ឡើង ក្នុង
Predicted text: ប្រាសាទ ស ំប ូរ ព្រៃ គ ុក មាន ទីតាំង ស្ថិតនៅក្នុង ភូមិ ស ំប ូរ  ឃុំ ស ំប ូរ  ស្រុក ប្រាសាទ ស ំប ូរ  ខេត្ត កំពង់ ធំ ។  ប្រាសាទ នេះមាន ច ម្ ងាយ
Predicted text: ប្រាសាទ ស ំប ូរ ព្រៃ គ ុក មាន ទីតាំង ស្ថិតនៅក្នុង ភូមិ ស ំប ូរ  ឃុំ ស ំប ូរ  ស្រុក ប្រាសាទ ស ំប ូរ  ខេត្ត កំពង់ ធំ ។  ប្រាសាទ នេះ ស្ថិតនៅ ច ម្
Predicted text: ប្រាសាទ ស ំប ូរ ព្រៃ គ ុក មាន ទីតាំង ស្ថិតនៅ ភូមិ ស ំប ូរ  ឃុំ ស ំប ូរ  ស្រុក ប្រាសាទ ស ំប ូរ  ខេត្ត កំពង់ ធំ ។  ប្រាសាទ នេះ កសាង ឡើង ក្នុង
Predicted text: ប្រាសាទ ស ំប ូរ ព្រៃ គ ុក មាន ទីតាំង ស្ថិតនៅក្នុង ភូមិ ស ំប ូរ  ឃុំ ស ំប ូរ  ស្រុក ប្រាសាទ ស ំប ូរ  ខេត្ត កំពង់ ធំ ។  ប្រាសាទ នេះ កសាង ឡើង នៅ
Predicted text: ប្រាសាទ នាង ខ្ម ៅ មាន ទីតាំង ស្ថិតនៅក្នុង ភូមិ ស ំប ូរ  ឃុំ ស ំប ូរ  ស្រុក ប្រាសាទ ស ំប ូរ  ខេត្ត កំពង់ ធំ ។  ប្រាសាទ នេះ កសាង ឡើង ក្នុង សម័យ ច េន
Predicted text: ប្រាសាទ ស ំប ូរ ព្រៃ គ ុក មាន

In [59]:
tokenizer.save("gpt2-wikipedia-khmer-tokenizer")